## Annual Report Data Engineering and Analysis

## Setup

In [ ]:
# Setup
import pandas as pd
import os


### Count of Permit by Category
*  Get Accela Permits data summarized by count of Reporting Category 
	Delete TMP files
	Establish Reporting Category
		Summary of Record Types
			Ken has lookup list for this
			
		Filter out Plan Revision
			A second lookup of the ID without -01 at the end and lookup list against the other File Number Record Type
			
	Count of Files by Reporting Category
		Just for current year


In [ ]:
df = pd.read_csv()